# Laborarbeit KI - CSP
**Belegung von Laboren für Projektpräsentationen**  
_Jona Scholz, Damian Roncevic_  

In diesem Labor soll für eine Präsentationswoche ein Präsentationsplan erstellt werden. In diesem Plan sollen alle Präsentationen aufgelistet werden, mit einer einmaligen _Tag-Zeit-Raum_-Kombination. Dafür müssen bestimmte **constraints** definiert werden (z.b. keine Doppelbelegung).

Folgender Ablauf wird genutzt, um dieses Problem zu lösen:
1. Benötigste Librarys installieren
2. Librarys importieren
3. Datensatz testweise ausgeben und analysieren
4. Umgebung definieren
5. Constraints definieren
6. Hilfs-Funktionen definieren
7. CSP initialisieren
8. Ausgabe formatieren
9. Durchführung
10. Analyse
11. Fazit

In [42]:
!pip install -r requirements.txt

Jetzt werden alle benötigten Bibliotheken importiert und ein Datensatz geladen, um einen ersten Überblick zu bekommen.

`Python-constraint` kann CSP's definieren und Lösungen dafür finden.  
`Pandas` wird genutzt, um die Datensätze zu laden und einfach zu manipulieren.  
`itertools` stellt hilfreiche Iterationsfunktionen zur Verfügung.  
`functools` stellt hilfreiche Funktionen zur Verfügungen. Hier wird `partial` genutzt, um Parameter direkt in eine Funktion zu integrieren. Das wird benötigt, um gewisse Constraints nutzen zu können.  
`time` wird für ein Timeout Constraint benutzt (später mehr).

In [43]:
from constraint import *
import pandas as pd
import itertools
from functools import partial
import time

pd.read_csv('./data/DS_CSP_1/pr_conf_004.txt', sep=';')

,Studiengang,Projektgruppen,Kommissionen
0,Informatik,4,3
1,Elektrotechnik,2,2
2,WIW,2,1
3,Maschinenbau,2,2
4,Data Science,1,3


Wir haben in diesem Datensatz 5 verschiedene Studiengänge. Jeder Studiengang hat eine Anzahl von Projektgruppen und Kommissionen.  
_"Die Kommissionen können nur Projekte ihres eigenen Studiengangs betreuen"_

## Umgebung definieren
Jetzt werden die Umgebungsvariablen definiert, wechle aus der Aufgabenstellung herauskommen:
- Tage: 5 (Mo. - Fr.)
- Zeitslots: 4
- Räume: 3
- Präsentationen pro Gruppe: 3
- Max Raumwechsel von Kommissionen: 1

Dazu werden Abkürzungen definiert, um die Ausgabe kompakt zu halten.

In [44]:
DAYS = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag']
SLOTS = ['S1', 'S2', 'S3', 'S4']
SLOT_TIMES = {'S1': '08:00 - 09:00', 'S2': '10:00 - 11:00', 'S3': '13:00 - 14:00', 'S4': '15:00 - 16:00'}
ROOMS = ['L1', 'L2', 'L3']
AMOUNT_PRESENTATIONS = 3
MAX_SWITCHES = 1

ACROS = {
    'Informatik': 'INF',
    'Data Science': 'DS',
    'Elektrotechnik': 'ET',
    'WIW': 'WIW',
    'Maschinenbau': 'MB',
    'Data Science': 'DS'
}

MAX_SLOTS = len(DAYS) * len(SLOTS) * len(ROOMS)
MAX_SLOTS

60

Es gibt insgesamt **60** Slots.

Jetzt müssen die jeweiligen constraints erstellt werden. Dafür wird hier jedes Constraint als Funktion definiert und später zum Projekt hinzugefügt.
## Constraints
Aus der Aufgabenstellungen lassen sich insgesamt __5__ harte Constraints sowie ein zusätzliches weiches Constraint ableiten.  

Die lassen sich wie folgt zusammenfassen:
- `unique_appointment`: Jede Präsentation braucht einen eindeutigen Termin (Tag, Slot, Raum).
- `group_clash`: Keine Projektgruppe kann Präsentation parallel halten. 
- `comitee_clash`: Keine Kommission kann mehreren Präsentation gleichzeitig zuhören.
- `max_switches`: Einer Prüfungskommission ist pro Tag __höchstens ein__ Wechsel des Laborraums gestattet.
- `rest_time`: Auf eine Präsentation im vierten Slot (15:00–16:00 Uhr) darf am unmittelbar folgenden Tag keine Präsentation im ersten Slot (08:00–09:00 Uhr) für dieselbe Gruppe folgen.

Ergänzt werden diese harten Bedinungen das weiche Constraint, nach dem frühere Slots bevorzugt genutzt werden sollen, um die Präsenzzeit der Kommissionen zu minimieren. Es dient der organisatorischen Effizienz, damit Prüfungskommissionen nicht länger als nötig im Haus gehalten werden.  
Es bedeutet, bei mehreren validen Lösungen, eine Lösung mit möglichst frühen Zeit-Slots zu bevorzugen.

Dieser wird programm technisch durch keine direkte Bedieungung gelöst, sondern durch __Value Ordering__ umgesetzt _(in den Hilfs-Funktionen)_.

In [45]:
def unique_appointment(d1: tuple, d2: tuple):
    """
    Constraint: Ensure that two presentations do not occur at the same time (day and slot) in the same room.
    """
    day1, slot1, room1, _ = d1
    day2, slot2, room2, _ = d2
    return not (day1 == day2 and slot1 == slot2 and room1 == room2)


def group_clash(d1: tuple, d2: tuple):
    """
    Constraint: Ensure that a group does not have overlapping presentations.
    Careful: The inputs need to be from the same group!
    """
    day1, slot1, _, _ = d1
    day2, slot2, _, _ = d2
    return not (day1 == day2 and slot1 == slot2)
    

def committee_clash(d1: tuple, d2: tuple):
    """
    Constraint: Ensure that a committee does not have overlapping presentations.
    """
    day1, slot1, _, committee1 = d1
    day2, slot2, _, committee2 = d2
    if committee1 == committee2:
        return not (day1 == day2 and slot1 == slot2)
    return True


def max_switches(*args, target_committee, target_day):
    """
    Constraint: Ensure that a committee does not have overlapping presentations.
    """
    # filter for target commitee and day
    relevant_sessions = []
    for val in args:
        day, slot, room, committee = val
        if day == target_day and committee == target_committee:
            relevant_sessions.append((slot, room))
    
    # only one or no sessions -> can't switch
    if len(relevant_sessions) <= 1:
        return True
    
    # sort by slot
    relevant_sessions.sort(key=lambda x: x[0])
    
    # count switches
    switches = 0
    current_room = relevant_sessions[0][1]
    
    for i in range(1, len(relevant_sessions)):
        next_room = relevant_sessions[i][1]
        if next_room != current_room:
            switches += 1
            current_room = next_room
            
    return switches <= MAX_SWITCHES


def rest_time(d1: tuple, d2: tuple):
    """
    Constraint: Ensure that when the group has the last slot, that it can have the first at the direct next day.
    """
    day1, slot1, _, _ = d1
    day2, slot2, _, _ = d2
    
    # Get indices for days and slots (0-4 for days, 0-3 for slots)
    day_index1 = DAYS.index(day1)
    day_index2 = DAYS.index(day2)
    slot_index1 = SLOTS.index(slot1)
    slot_index2 = SLOTS.index(slot2)
    
    # Day1 is day n  (slot 4) and Day2 is day n+1 (slot 1)
    if day_index1 + 1 == day_index2 and slot_index1 == 3 and slot_index2 == 0:
        return False 
    
    # Day1 is day n+1 (slot 1) and Day2 is day n (slot 4)
    if  day_index1 == day_index2 + 1 and slot_index2 == 0 and slot_index1 == 3:
        return False 
    
    return True

## Hilfsfunktionen
Hier werden Hilfsfunktionen definert, für die spätere Variableninitialisierung.

### getDomain():  
Erstellt die Domain für den jeweiligen Studiengang. Eine Domain ist eine Liste aus `(TAG, ZEITSLOT, RAUM, KOMISSION)`-Terminen.  
Damit das weiche constraint _("Prüfungskommissionen sollten an einem Tag nicht länger als nötig im Haus gehalten werden,
d.h. wenn ein früherer Präsentationsslot möglich ist, so ist dieser zu nutzen")_ erreicht wird, werden die Termine innerhalb der Domänen zuerst erstellt und dann sortiert.

Das Sortieren läuft folgendermaßen ab:

1. Die Termine abfallend nach Zeitslots sortieren.
2. Termine abfallend nach den Tagen sortieren.
3. Termine abfallend nach den Räumen sortieren.

Durch eine gewichtete Sortierung der Domain entsteht eine Priorisierung, die Zeitslots, Tage und Räume so balanciert, dass Präsentationen bevorzugt in frühen Zeitfenstern eingeplant werden.
Der __Backtracking-Algorithmus__ nutzt diese wertbasierte Heuristik (__Value Ordering__), um eine Lösung zu generieren, die zeitlich optimiert ist und gleichzeitig eine gleichmäßige Lastverteilung über die gesamte Woche und alle Laborressourcen sicherstellt.

Einfaches Beispiel:
`[("Wen", "S2"), ("Tue", "S2"), ("Mon", "S2"), ..., ("Wen", "S1"), ("Tue", "S1"), ("Mon", "S1")]`


### getVariables():  
Erstellt für den jeweiligen Studiengang eine Liste von allen Gruppen, mit ihren 3 Präsentationen. Die Variablennamen sind folgendermaßen aufgebaut: `{STUDIENGANG}-{GRUPPE}_P{PRÄSENTATION}`. Dadurch werden eindeutige Variablennamen garantiert.

In [46]:
def getDomain(study_course: str, amount_committees: int) -> list:
    """
    Create the domian for the presentation variables, based on days, slots, rooms and committees.
    
    Returns: a list of tuples (day, slot, room, committee)
    """
    domain = []
    for day in DAYS:
        for slot in SLOTS:
            for room in ROOMS:
                for committee in range(amount_committees):
                    domain.append((day, slot, room, f"{study_course}_K{committee}"))
                    
    # soft constraint to use earlier slots first
    domain.sort(key=lambda x: (SLOTS.index(x[1]), DAYS.index(x[0]), ROOMS.index(x[2])), reverse=True)
    return domain

def getVariables(study_course: str, amount_groups: int) -> list:
    """
    Defines all variables for a given study course and amount of groups.
    
    Returns: a list of variable names
    """
    variables = []
    for group in range(1, amount_groups + 1):
        for p in range(1, AMOUNT_PRESENTATIONS + 1):
            var_name = f"{study_course}-{group}_P{p}"  # Unique name - e.g. "INF_1_P1"
            variables.append(var_name)
    return variables

## Problem definieren
Hier wird eine Funktion definiert, welche das Problem repräsentiert. Dabei werden die Variablen, für die jeweiligen Gruppen, mit ihren Domänen initialisiert und die Constraints für die jeweiligen Variablen hinzugefügt.

Zuerst wird für jede Präsentation von jeder Gruppe eine Varibale hinzugefügt, mit der Domäne `(Tag, Slot, Raum, Komissionen)`.  
Danach wird überprüft, ob das Problem lössbar ist.  
Als nächstes werden die Bedingungen für die jeweiligen Variablen erstellt.

Die Bedingung `unique_appointment` wird für jede Porjektgruppe einzeln angewandt und stellt sicher, dass jede Präsentation auf eindeutige Termine (Tag, Slot, Raum) verteilt werden.

Die Bedingungen `group_clash` und `rest_time` werden auf die jeweiligen Gruppen definiert. Dafür wird über alle Gruppen iteriert und diese Constraints dann nur innerhalb der Gruppen eingeführt.

Die letzten Constraints sind die Kommissionsabhängigen (`committee_clash` und `max_switches`). Hierzu wird über die Studiengänge iteriert, um auf die Gruppen in diesem zuzugreifen (weil Kommissionen nur in ihrem Studiengang prüfen dürfen). Dann werden die Bedinugen auf die Studiengangsgruppen angewendent.  
Für `max_switches` wird nochmal über die Prüfungskommissionen iteriert, um dann die Zielkommission und den Zieltag definieren zu können.

Zum Schluss wird nach einem Ergebnis gesucht und zurückgegeben, falls es eine Lösung gibt.

In [54]:
solvers = {
    'backtracking': BacktrackingSolver,
    'min-conflicts': MinConflictsSolver,
    'recursive-bt': RecursiveBacktrackingSolver
}

def presentation_problem(input_data: pd.DataFrame, 
                         solver: str = 'backtracking') -> dict:
      
    problem = Problem(solver=solvers[solver]())
    
    # --- Define variables ---
    variables = []
    vars_by_course: dict = {}
    for study in input_data.itertuples():
        # get variables and domain for each study course
        vars = getVariables(ACROS[study[1]], study[2])
        domain = getDomain(ACROS[study[1]], study[3])
        variables.extend(vars)
        vars_by_course[ACROS[study[1]]] = vars
        
        problem.addVariables(vars, domain)
    
    # --- little check ---
    if len(variables) > MAX_SLOTS:
        print("Too many variables for available slots!")
        return None
    elif len(variables) == MAX_SLOTS:
        print("Warning: Variables equal to available slots, no freedom in scheduling.")
    
    
    # --- Define Constraints ---
    
    # Unique appointments - on all variables
    for i in range(len(variables)):
        for j in range(i + 1, len(variables)):
            problem.addConstraint(unique_appointment, (variables[i], variables[j]))
    
    # group constraints - group clash and rest time
    for idx in range(0, len(variables), 3):
        # split into list of group presentations - can split by AMOUNT_PRESENTATIONS
        group_vars = variables[idx:idx + AMOUNT_PRESENTATIONS]

        # iter over all group combinations
        for g1, g2 in itertools.combinations(group_vars, 2):
            # Add group clash constraint       
            problem.addConstraint(group_clash, (g1, g2))

            # Add rest time constraint
            problem.addConstraint(rest_time, (g1, g2))

    # comitee constraints - comitee clash and max switches
    for course, vars in vars_by_course.items():
        full_course_name = next((key for key,value in ACROS.items() if value == course), None)
        
        # comitee clash - for each variable pair in a study course
        for var1, var2 in itertools.combinations(vars, 2):
            problem.addConstraint(committee_clash, (var1, var2))
            
        # max switches - for each committee and day
        # go over all committees
        for i in range(input_data.loc[input_data['Studiengang'] == full_course_name, 'Kommissionen'].values[0]):
            commitee = f"{course}_K{i}"   # Need the same format as in domain
            
            # for each day add the constraint
            for day in DAYS:
                # partial function to add traget committee and day into the max_switches function
                problem.addConstraint(partial(max_switches, target_committee=commitee, target_day=day), vars)
            
            
    # --- Solve the problem --- 
    return problem.getSolution()
    

## Ausgabe formatieren
Um nicht ein Dictionary anschauen zu müssen, wird eine anschauliche Tabelle erstellt mithilfe von der Pandas `pivot_table`. Dabei repräsentieren die Spalten die jeweiligen Tage und die Zeilen die Räume mit ihren Zeitslots. 

In [48]:
def schedule_table(solution: dict) -> pd.DataFrame:
    """
    Creates a table with slots as rows and days as columns.
    Cell = "Group Committee Room"
    """
    
    # create formatted data
    formatted_data = []
    for var, (day, slot, room, committee) in solution.items():
        group, presentation = var.split('_')
        label = f"{group} ({presentation}) - [{committee}]"    # e.g. "INF-1 (P1) - [INF_K0]"
        
        formatted_data.append({
            'Tag': day,
            'Slot': SLOT_TIMES.get(slot, slot), # convert slot to time range
            'Raum': room,
            'Belegung': label
        })
        
    df = pd.DataFrame(formatted_data)
    
    # create table for display
    table = df.pivot_table(
        index=['Raum', 'Slot'], 
        columns='Tag', 
        values='Belegung', 
        aggfunc='first'
    )
    # Reindex to ensure all days and slots/rooms are present and fill missing
    table = table.reindex(columns=DAYS, index=pd.MultiIndex.from_product(
        [ROOMS, [SLOT_TIMES[s] for s in SLOTS]], names=['Raum', 'Uhrzeit'])).fillna('---')
    
    return table

## Durchführung
Um das Problem für die jeweiligen Datensätze zu lösen wird eine Funktion definiert, welche die Inputdaten bekommt und mit diesen das Problem startet und ein Präsentationsplan ausgibt, wenn das Problem lösbar ist.

- `precheck`: Schaut ob es theoretisch möglich ist das problem zu lösen. Dabei schaut es ob es genug Raum / Slot Kapazitäten gibt und ob die Komissionskapazitäten ausreichen.
- `exe`: Fürht das Problem aus und gibt wenn möglich ein Präsentationsplan zurück.

In [51]:
from typing import Literal

def precheck(data: pd.DataFrame, slots_per_week: int = len(SLOTS) * len(DAYS)) -> bool:
    """
    Checks whether the scheduling problem is theoretically solvable with given data.
    - total presentations do not exceed available slots
    - presentations per study course do not exceed committee capacity
    """
    total_presentations = data['Projektgruppen'].apply(lambda x: x * AMOUNT_PRESENTATIONS).sum()
    
    # Check total presentations against available slots
    if total_presentations > MAX_SLOTS:
        print(f"Precheck failed: Total presentations ({total_presentations}) exceed available slots ({MAX_SLOTS}).")
        return False
    
    # Check presentations against committee capacity
    for study in data.itertuples():
        total_study_presentations = study[2] * AMOUNT_PRESENTATIONS
        
        committee_capacity = study[3] * slots_per_week  # not with amount of rooms,because committees are not multipliable
        if total_study_presentations > committee_capacity:
            print(f"Precheck failed: Study course '{study[1]}' requires {total_study_presentations} presentations, but committee capacity is {committee_capacity}.")
            return False
    # No issues found, problem theoretically solvable
    return True


def exe(data: pd.DataFrame, solver: 
    Literal["backtracking", "recursive-bt", "min-conflicts"] = "backtracking"):
    
    # Precheck - if solution is possible
    precheck_result = precheck(data)
    if not precheck_result:
        return None
    
    # Solve CSP with timeout
    solution = presentation_problem(data, solver=solver) 
    
    if solution is None:
        print("No solution found.")
        return
    
    return schedule_table(solution)


### Datensatz - 001

In [58]:
data = pd.read_csv('./data/DS_CSP_1/pr_conf_001.txt', sep=';')


exe(data, solver='min-conflicts')

Tag                                Montag               Dienstag  \
Raum Uhrzeit                                                       
L1   08:00 - 09:00    MB-2 (P1) - [MB_K0]    DS-1 (P1) - [DS_K0]   
     10:00 - 11:00    ET-4 (P3) - [ET_K0]    MB-3 (P1) - [MB_K0]   
     13:00 - 14:00  WIW-1 (P2) - [WIW_K0]  INF-5 (P1) - [INF_K1]   
     15:00 - 16:00  INF-7 (P3) - [INF_K1]  INF-6 (P3) - [INF_K2]   
L2   08:00 - 09:00  INF-2 (P2) - [INF_K0]  INF-3 (P3) - [INF_K1]   
     10:00 - 11:00  WIW-1 (P3) - [WIW_K0]  INF-4 (P2) - [INF_K1]   
     13:00 - 14:00  INF-8 (P3) - [INF_K0]  WIW-1 (P1) - [WIW_K0]   
     15:00 - 16:00  INF-4 (P3) - [INF_K0]    MB-4 (P1) - [MB_K1]   
L3   08:00 - 09:00    MB-5 (P1) - [MB_K1]    MB-1 (P2) - [MB_K0]   
     10:00 - 11:00  INF-6 (P1) - [INF_K1]    ET-2 (P3) - [ET_K1]   
     13:00 - 14:00  INF-2 (P1) - [INF_K1]  INF-7 (P1) - [INF_K2]   
     15:00 - 16:00  INF-2 (P3) - [INF_K2]  INF-4 (P1) - [INF_K0]   

Tag                              Mittwoch             Donnerstag  \
Raum Uhrzeit                                                       
L1   08:00 - 09:00    DS-2 (P3) - [DS_K0]    MB-4 (P3) - [MB_K1]   
     10:00 - 11:00    MB-3 (P2) - [MB_K0]    DS-2 (P1) - [DS_K0]   
     13:00 - 14:00    ET-1 (P3) - [ET_K0]  INF-3 (P2) - [INF_K0]   
     15:00 - 16:00    ET-3 (P1) - [ET_K0]  INF-5 (P3) - [INF_K2]   
L2   08:00 - 09:00  INF-1 (P1) - [INF_K1]  INF-5 (P2) - [INF_K1]   
     10:00 - 11:00    ET-1 (P1) - [ET_K1]  INF-1 (P3) - [INF_K1]   
     13:00 - 14:00  INF-8 (P1) - [INF_K2]    DS-1 (P3) - [DS_K0]   
     15:00 - 16:00    MB-2 (P3) - [MB_K1]    ET-1 (P2) - [ET_K0]   
L3   08:00 - 09:00    MB-1 (P1) - [MB_K1]  INF-7 (P2) - [INF_K2]   
     10:00 - 11:00    ET-2 (P2) - [ET_K0]    MB-2 (P2) - [MB_K1]   
     13:00 - 14:00    ET-3 (P2) - [ET_K1]    MB-5 (P2) - [MB_K1]   
     15:00 - 16:00    ET-4 (P2) - [ET_K1]    MB-4 (P2) - [MB_K0]   

Tag                               Freitag  
Raum Uhrzeit                               
L1   08:00 - 09:00    MB-1 (P3) - [MB_K0]  
     10:00 - 11:00    ET-3 (P3) - [ET_K1]  
     13:00 - 14:00    MB-5 (P3) - [MB_K0]  
     15:00 - 16:00  INF-3 (P1) - [INF_K2]  
L2   08:00 - 09:00    ET-4 (P1) - [ET_K0]  
     10:00 - 11:00    DS-1 (P2) - [DS_K0]  
     13:00 - 14:00    DS-2 (P2) - [DS_K0]  
     15:00 - 16:00    MB-3 (P3) - [MB_K1]  
L3   08:00 - 09:00  INF-6 (P2) - [INF_K0]  
     10:00 - 11:00  INF-8 (P2) - [INF_K2]  
     13:00 - 14:00  INF-1 (P2) - [INF_K2]  
     15:00 - 16:00    ET-2 (P1) - [ET_K1]

#### Interpretation der Lösung
Das CSP-Modell hat eine valide Lösung für den Datensatz `001` gefunden. Dabei wurden alle harten Bedingungen erfüllt, da der Plan voll ausgelastet ist, kommt das weiche Constraint nicht wirklich zum Einsatz. Jedoch könnte die Verteilung besser sein, dass eine Gruppe oder Kommission nicht mehrere Slots warten muss auf die nächste Präsentation (z.b. ET_K1 am Freitag: 10:00-11:00 Uhr und dann erst wieder von 15:00-16:00 Uhr).

Es wurde mir mit dem Solver `min-conflicts` eine Lösung in absehbarer Zeit gefunden.

Der __Backtracking__ Algorithmus versucht einen den Suchbaum von oben bis unten aufzubauen. Das Problem ist bei einer vollen Auslastung, dass fast alle Zweige in eine Sackgasse führen und der Algorithmus in den letzten Abschnitten ein Constraint verletzt muss dann stark gebacktrackt werden.  
Bei 60 Präsentationen ist die Anzahl der Kombinationen extrem hoch ($domain^{Presentation}$).

Die Strategie von __Min-Conflicts__ ist anders. Er startet mit einer wahrscheinlich falschen Kombination von Variablen und sucht sich dann Variablen raus, die Konflikte verursachen und ändert den Wert um Anzahl der Konflikte zu minimieren. Dadurch findet dieser oft schneller Lösungen in einem dichten Problem.  
*Durch die zufällige Zuweisung von Variablen, würde dieser Algorithmus bei einer geringen Auslastung das weiche Constraint ignorieren.*

### Datensatz - 004

In [ ]:
data = pd.read_csv('./data/DS_CSP_1/pr_conf_004.txt', sep=';')

exe(data)

Tag                                Montag               Dienstag  \
Raum Uhrzeit                                                       
L1   08:00 - 09:00  INF-1 (P1) - [INF_K2]  INF-1 (P2) - [INF_K2]   
     10:00 - 11:00  INF-4 (P3) - [INF_K2]    ET-2 (P1) - [ET_K1]   
     13:00 - 14:00  WIW-1 (P2) - [WIW_K0]  WIW-1 (P3) - [WIW_K0]   
     15:00 - 16:00                    ---                    ---   
L2   08:00 - 09:00  INF-2 (P1) - [INF_K1]  INF-2 (P2) - [INF_K1]   
     10:00 - 11:00    DS-1 (P3) - [DS_K2]    MB-1 (P1) - [MB_K1]   
     13:00 - 14:00                    ---                    ---   
     15:00 - 16:00                    ---                    ---   
L3   08:00 - 09:00  INF-3 (P1) - [INF_K0]  INF-3 (P2) - [INF_K0]   
     10:00 - 11:00    ET-1 (P3) - [ET_K1]    MB-2 (P1) - [MB_K0]   
     13:00 - 14:00                    ---                    ---   
     15:00 - 16:00                    ---                    ---   

Tag                              Mittwoch             Donnerstag  \
Raum Uhrzeit                                                       
L1   08:00 - 09:00  INF-1 (P3) - [INF_K2]  INF-4 (P1) - [INF_K2]   
     10:00 - 11:00    ET-2 (P2) - [ET_K1]    ET-2 (P3) - [ET_K1]   
     13:00 - 14:00  WIW-2 (P1) - [WIW_K0]  WIW-2 (P2) - [WIW_K0]   
     15:00 - 16:00                    ---                    ---   
L2   08:00 - 09:00  INF-2 (P3) - [INF_K1]    DS-1 (P1) - [DS_K2]   
     10:00 - 11:00    MB-1 (P2) - [MB_K1]    MB-1 (P3) - [MB_K1]   
     13:00 - 14:00                    ---                    ---   
     15:00 - 16:00                    ---                    ---   
L3   08:00 - 09:00  INF-3 (P3) - [INF_K0]    ET-1 (P1) - [ET_K1]   
     10:00 - 11:00    MB-2 (P2) - [MB_K0]    MB-2 (P3) - [MB_K0]   
     13:00 - 14:00                    ---                    ---   
     15:00 - 16:00                    ---                    ---   

Tag                               Freitag  
Raum Uhrzeit                               
L1   08:00 - 09:00  INF-4 (P2) - [INF_K2]  
     10:00 - 11:00  WIW-1 (P1) - [WIW_K0]  
     13:00 - 14:00  WIW-2 (P3) - [WIW_K0]  
     15:00 - 16:00                    ---  
L2   08:00 - 09:00    DS-1 (P2) - [DS_K2]  
     10:00 - 11:00                    ---  
     13:00 - 14:00                    ---  
     15:00 - 16:00                    ---  
L3   08:00 - 09:00    ET-1 (P2) - [ET_K1]  
     10:00 - 11:00                    ---  
     13:00 - 14:00                    ---  
     15:00 - 16:00                    ---

#### Interpretation der Lösung
Bei dieser Lösung für den Datensatz `004` ist die Auslastung sehr moderat mit ca. 50%. Es werden alle harten Bedingungen erfüllt und die weiche auch. Jedoch könnte die Auslastung noch besser verteilt werden (siehe Freitag). Dort könnte die Verteilung auf insgesamt 2 Zeitslots reduziert werden, indem eine WIW-Gruppe ihre Präsentation im ersten Slot macht und die andere im zweiten.

### Datensatz - 005

In [ ]:
data = pd.read_csv('./data/DS_CSP_1/pr_conf_005.txt', sep=';')

exe(data)

Tag                                Montag               Dienstag  \
Raum Uhrzeit                                                       
L1   08:00 - 09:00  INF-1 (P1) - [INF_K2]  INF-1 (P2) - [INF_K2]   
     10:00 - 11:00  INF-4 (P3) - [INF_K2]  WIW-2 (P1) - [WIW_K2]   
     13:00 - 14:00    ET-2 (P2) - [ET_K1]    ET-2 (P3) - [ET_K1]   
     15:00 - 16:00                    ---                    ---   
L2   08:00 - 09:00  INF-2 (P1) - [INF_K1]  INF-2 (P2) - [INF_K1]   
     10:00 - 11:00  INF-5 (P3) - [INF_K1]    MB-1 (P1) - [MB_K2]   
     13:00 - 14:00    DS-1 (P2) - [DS_K0]    DS-1 (P3) - [DS_K0]   
     15:00 - 16:00                    ---                    ---   
L3   08:00 - 09:00  INF-3 (P1) - [INF_K0]  INF-3 (P2) - [INF_K0]   
     10:00 - 11:00  WIW-1 (P3) - [WIW_K2]    ET-1 (P1) - [ET_K1]   
     13:00 - 14:00                    ---                    ---   
     15:00 - 16:00                    ---                    ---   

Tag                              Mittwoch             Donnerstag  \
Raum Uhrzeit                                                       
L1   08:00 - 09:00  INF-1 (P3) - [INF_K2]  INF-4 (P1) - [INF_K2]   
     10:00 - 11:00  WIW-2 (P2) - [WIW_K2]  WIW-2 (P3) - [WIW_K2]   
     13:00 - 14:00                    ---                    ---   
     15:00 - 16:00                    ---                    ---   
L2   08:00 - 09:00  INF-2 (P3) - [INF_K1]  INF-5 (P1) - [INF_K1]   
     10:00 - 11:00    MB-1 (P2) - [MB_K2]    MB-1 (P3) - [MB_K2]   
     13:00 - 14:00                    ---                    ---   
     15:00 - 16:00                    ---                    ---   
L3   08:00 - 09:00  INF-3 (P3) - [INF_K0]  WIW-1 (P1) - [WIW_K2]   
     10:00 - 11:00    ET-1 (P2) - [ET_K1]    ET-1 (P3) - [ET_K1]   
     13:00 - 14:00                    ---                    ---   
     15:00 - 16:00                    ---                    ---   

Tag                               Freitag  
Raum Uhrzeit                               
L1   08:00 - 09:00  INF-4 (P2) - [INF_K2]  
     10:00 - 11:00    ET-2 (P1) - [ET_K1]  
     13:00 - 14:00                    ---  
     15:00 - 16:00                    ---  
L2   08:00 - 09:00  INF-5 (P2) - [INF_K1]  
     10:00 - 11:00    DS-1 (P1) - [DS_K0]  
     13:00 - 14:00                    ---  
     15:00 - 16:00                    ---  
L3   08:00 - 09:00  WIW-1 (P2) - [WIW_K2]  
     10:00 - 11:00                    ---  
     13:00 - 14:00                    ---  
     15:00 - 16:00                    ---

#### Interpretation der Lösung
Auch für diese Lösung vom Datensatz `005` ist die Auslastung sehr moderat mit ca. 50%. Es werden wieder alle harten Bedingungen erfüllt und die weiche auch. Jedoch ist es hier die Verteilung besser. Am Freitag wird im Raum `L3` nur eine Präsentation gehalten, aber besser als für den anderen Datensatz.

## Fazit
In dieser Laborarbeit wurde ein CSP-Modell erstellt, um ein Präsentationsplan für verschiedene Projektgruppen aus verschiedenen Studiengängen zu generieren. Dabei wurden die harten Bedingungen aus der Aufgabenstellung erfüllt und die zusätzliche weiche Bedingung ebenfalls. Diese könnte jedoch noch mehr verfeinert werden, damit die Präsentationen noch flacher über alle Tage gehalten werden.  
Um das zu erreichen könnte ein zusätzliches hartes Constraint helfen indem ein Tageslimit für Präsentationen gesetzt wird. Das würde den Solver zwingen Ausreißer (wie in Lösung für `004`) in die freien Plätze der anderen Räume zu schieben. Das Limit wäre dabei etwas höher als die durchschnittliche Anzahl von Präsentationen am Tag (z.b. $limit = amountPresentations / amountDays + 2$).

Methodik und Solver-Wahl (Der wichtigste Teil)

Erkenntnisse aus der Implementierung

Ausblick


_„Die erfolgreiche Modellierung des Stundenplanproblems als Constraint Satisfaction Problem (CSP) zeigt die Mächtigkeit heuristischer Suchverfahren. Besonders die Erkenntnis, dass bei maximaler Ressourcenauslastung die Wahl des Solvers (Min-Conflicts statt Backtracking) entscheidend für die Terminierung des Algorithmus ist, unterstreicht die theoretischen Grenzen systematischer Suchbäume in NP-vollständigen Problemen. Das Ergebnis ist ein robuster und fairer Plan, der sowohl studentische Belange (Ruhezeiten) als auch dozentenrelevante Effizienz (Raumwechsel, frühe Slots) optimal vereint.“_